# Hello there! We heard you're planning a HackWeek!
Collaboratr is a tool that will help connect hackers with knowledge with people who want to learn from them. To use this tool, simply follow the steps contained in this notebook.

In [87]:
%matplotlib inline

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from operator import truediv
from nxpd import draw
from collections import Counter
import itertools
import random

import collaboratr
reload(collaboratr)

<module 'collaboratr' from 'collaboratr.pyc'>

# Step 1 Create a Google Form with the following four questions:
    1. What is your name? [text entry]
    2. What is your email? [text entry]
    3. What can you help others with? [checkboxes with possible topics]
    4. What do you want to collaborate on or learn about? [same topic checkboxes]
    
After receiving the responses from your attendees, load up the CSV of responses from the google form by running the cell below (you'll have to change the path to your own CSV)

In [88]:
# Retrieve data from Google Sheet and parse using pandas dataframe
data = pd.read_csv("https://raw.githubusercontent.com/benelson/collaboratr/master/collaboratr.csv")
data = data.replace(np.nan,' ', regex=True)
users = data['1. What is your name?']
emails = data['2. What is your email?']
teach = data['3. What can you help others with?']
learn = data['4. What do you want to collaborate on or learn about?']

# Step 2: Determine your desired breakout threshold
Breakout sessions are a great way to help your attendees learn about topics that many are interested in, particularly those topics where the number of learners is much larger than the number of potential teachers. The following parameters govern the establishment of breakout sessions:

* `BREAKOUT_REQ_RATIO` sets a minimum ratio of learners to teachers for a breakout to be established. If this is $\leq 1$, you may prefer to have individual discussions in lieu of a breakout.
* `BREAKOUT_REQ_FRACTION` will establish a breakout session for any topic where the fraction of people expressing desire to learn a topic is greater than this fraction of total attendees (survey respondents). If a large fraction of your hackers want to learn about a topic, a breakout may be more efficient than individual discussions.

In [89]:
BREAKOUT_REQ_RATIO = 2
BREAKOUT_REQ_FRACTION = 0.2

# Step 3: Determine breakout topics
The following cell will compute the necessary breakout topics based on the parameters specified above. We determine these before assigning individual topic pairings.

In [90]:
breakouts = collaboratr.find_breakouts(users,teach,learn,BREAKOUT_REQ_RATIO,BREAKOUT_REQ_FRACTION)

In [91]:
print 'It looks like you\'ll be having breakouts in:'
for i,b in enumerate(breakouts): print str(i+1)+'.',b

It looks like you'll be having breakouts in:
1. GAIA DRI
2. packaging (Python)
3. gaussian processes
4. autodiff


# Step 4: Compute the Overall Graph of Topic Pairings
These cells will produce an overall social graph of your hackathon, with arrows pointing from a learner to a recommended teacher to consult on a topic.

In [92]:
G = nx.DiGraph()

In [93]:
# Insert users in graphs
for u,e,l,t in zip(users, emails, learn, teach):
    collaboratr.insert_node(G,u, email=e, learn=l.split(';'), teach=t.split(';'))

In [94]:
participants = G.nodes(data=True)

In [95]:
assign = collaboratr.assign_users(G,participants,breakouts)

In [96]:
G_new = nx.DiGraph(bgcolor='white')
G_new.add_nodes_from(participants, style='filled', fillcolor='black', 
                     fontcolor='white', fontname='sans-serif', fontsize=24)
G_new.add_node('breakout', shape='rectangle', width=4, height=1, style='filled', fillcolor='black',
              fontcolor='white', fontname='sans-serif', fontsize=30)

In [97]:
for p in G.nodes():
    mappings = assign[p]
    for m in mappings.keys():
        G_new.add_edge(p, mappings[m], label=m, style='dotted', fontcolor='red', fontsize=20,
                      fontname='sans-serif')

In [98]:
draw(G_new, filename='Resources/0.svg')

'Resources/0.svg'

# Step 5: Draw Individually Customized Graphs
These cells will produce sub-graphs for each learner, so that each attendee can easily determine who they should consult during the hack week.

In [99]:
participants = G_new.nodes()
connections = G_new.edges(data=True)

In [100]:
index = collaboratr.drawIndividualGraphs(participants,connections)

In [101]:
#needed to create webpage below
obj = {}
obj['index'] = index
obj['participants'] = [p.split(' ')[0] for p in participants]

# Step 6: Creating a HackWeek Reference Web Page
The following cells will create a webpage with both the overall and individual social graphs. If you host the html file on your webpage, attendees will be able to click on their name and find their customized learning graph.

In [102]:
TEMPLATE1 = collaboratr.createWebPage(obj)

In [103]:
with open('display.html', 'w') as file:
    file.write(TEMPLATE1.render(obj=obj))

# Step 7: Create a Breakouts Web Page
This separate webpage will allow teachers and learners to collaboratively arrange breakout sessions during the hack week.

In [104]:
participants = G.nodes(data=True)
TEMPLATE2 = collaboratr.createBreakoutPage(participants,breakouts)

In [105]:
with open('breakouts.html', 'w') as file:
    file.write(TEMPLATE2.render(objs=objs))

# Step 8: Enjoy streamlined learning at your hack week!